In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

Load files

In [ ]:
notes = pd.read_csv('/sc-resources/dh-mimic/mimic_iv_2_2/note/discharge.csv')
admissions = pd.read_csv('/sc-resources/dh-mimic/mimic_iv_2_2/hosp/admissions.csv.gz')
patients = pd.read_csv('/sc-resources/dh-mimic/mimic_iv_2_2/hosp/patients.csv.gz')
readm_info = pd.read_csv('/home/wite10/single_notes_readmission_info.csv')

In [ ]:
patients.columns

In [ ]:
readm_info = readm_info[['hadm_id','subject_id','visit_no','thirty_day_readmission']]

## Merge files (readm_info ensures that only notes with known ground truth are used)

In [ ]:
info_notes = pd.merge(readm_info, notes, how = 'left', on = ['subject_id','hadm_id'], validate = 'one_to_one')

In [ ]:
info_notes_admissions = pd.merge(info_notes, admissions, how = 'inner', on = ['subject_id','hadm_id'], 
                       validate = 'one_to_one')

In [ ]:
merged_full = pd.merge(info_notes_admissions, patients, how = 'left', on = ['subject_id'], 
                    validate = 'many_to_one')

In [ ]:
print(readm_info.shape)
print(info_notes.shape)
print(info_notes_admissions.shape)
print(merged_full.shape)

Inspect data

In [ ]:
#merged_full['race'].value_counts()

In [ ]:
#merged_full['anchor_age'].value_counts()

In [ ]:
#merged_full['gender'].value_counts()

### Compute series of patients with notes

In [ ]:
patients_w_notes = readm_info['subject_id'].drop_duplicates(keep = 'first')

In [ ]:
patients_w_notes.shape

## Race filtering

### Remove patients with ambiguous race info

Races of all patients with notes

In [ ]:
patients_w_notes_races = pd.merge(patients_w_notes, admissions[['subject_id','race']], 
                                  on = 'subject_id', how = 'inner').drop_duplicates()

--> Examples for ambiguous race info:

In [ ]:
#print(patients_w_notes_races[patients_w_notes_races['subject_id'] == 13166511])
#print(patients_w_notes_races[patients_w_notes_races['subject_id'] == 11139232])

In [ ]:
#patients_w_notes_races['subject_id'].value_counts()

Remove every patient whose race changes across notes

In [ ]:
race_counter = pd.DataFrame(patients_w_notes_races['subject_id'].value_counts()).reset_index()
unambiguous_patients = list(race_counter[race_counter['count'] == 1]['subject_id'])

In [ ]:
merged_unambiguous = merged_full[merged_full['subject_id'].isin(unambiguous_patients)].copy()

### Remove 'unknown' etc. categories from 'race'

In [ ]:
excluded_races = ['OTHER', 'UNKNOWN', 'PATIENT DECLINED TO ANSWER', 'UNABLE TO OBTAIN', 'MULTIPLE RACE/ETHNICITY']
merged_limited_races = merged_unambiguous[~merged_unambiguous['race'].isin(excluded_races)]

In [ ]:
#merged_unambiguous['race'].value_counts()

In [ ]:
merged_limited_races.shape

### Remove gender-race-subgroups with less than 100 patients

In [ ]:
merged = merged_limited_races.groupby(['gender','race']).filter(lambda x: len(x) >= 100)

In [ ]:
#merged_limited_races[merged_limited_races['gender']=='M']['race'].value_counts()

In [ ]:
merged.shape

## Compute actual age of patient for every note

Convert admittime and dischtime to date

In [ ]:
#type(merged.loc[0,'dischtime'])

In [ ]:
merged['admittime'] = pd.to_datetime(merged['admittime'], errors = 'raise')
merged['dischtime'] = pd.to_datetime(merged['dischtime'], errors = 'raise')

In [ ]:
print(merged["dischtime"].dtype)
print(merged["dischtime"].head())

Compute actual_age

In [ ]:
merged.loc[:, 'actual_age'] = merged.loc[:,'dischtime'].dt.year - merged.loc[:,'anchor_year'] + merged.loc[:,'anchor_age']

In [ ]:
merged[['subject_id', 'dischtime', 'actual_age', 'anchor_age', 'anchor_year']]

## Age distribution over all notes (after race filtering)

Plot empirical age distribution over all notes

In [ ]:
age_dist_exact_prob = pd.DataFrame(merged['actual_age'].value_counts(normalize = True))
age_dist_exact_prob = age_dist_exact_prob.reset_index()

In [ ]:
plt.figure(figsize = (10,4)) 
plt.xticks(fontsize = 10, rotation = 45)
age_barplot = sns.barplot(x = 'actual_age', y = 'proportion', data = age_dist_exact_prob, color = 'b')
plt.ylabel('Proportion of Notes', fontsize = 14)
plt.xlabel('Age at Discharge Time', fontsize = 14)
plt.yticks(fontsize = 11)
ticks = plt.xticks(fontsize = 11)[0] 
plt.xticks(ticks[2::5])  
plt.tight_layout()
age_barplot_fig = age_barplot.get_figure()
age_barplot_fig.savefig('/home/wite10/images/agedist_complete_bar.svg')

full dataset as reference for age distribution

In [ ]:
#age_dist_exact = pd.DataFrame(merged['actual_age'].value_counts())
#age_dist_exact = age_dist_exact.reset_index()
#age_dist_exact.rename(columns = {'count': 'age_count'}, inplace = True)

- count how often a specific age appears in the merged data set
- count how often a specific age appears in all race-gender-subgroups
- compute sampling weights according to age distribution of merged by dividing the overall age count (weight of the age in merged) by the weight (count) this age-group has in the gender-race subgroup

In [ ]:
merged_weighted = merged.copy()
merged_weighted['age_count'] = merged_weighted.groupby('actual_age').transform('size')
merged_weighted['gender_race_age_count'] = merged_weighted.groupby(['actual_age', 'race', 'gender']).transform('size')
merged_weighted['weight'] = merged_weighted['age_count']/merged_weighted['gender_race_age_count']

In [ ]:
#merged_weighted = pd.merge(merged, age_dist_exact, on = 'actual_age')

In [ ]:
#merged_weighted[['subject_id','race','gender','actual_age','age_count','gender_race_age_count', 'weight']][merged_weighted['actual_age']==22]

### Difference in age and race distribution by gender

In [ ]:
grouped_by_gender = merged_weighted.groupby('gender')
print(merged_weighted['actual_age'].quantile([0,0.25, 0.5, 0.75,1]))
print(grouped_by_gender.get_group('F')['actual_age'].quantile([0,0.25, 0.5, 0.75,1]))
print(grouped_by_gender.get_group('M')['actual_age'].quantile([0,0.25, 0.5, 0.75,1]))
print(sns.boxplot(data = merged_weighted, x = 'gender', y = 'actual_age'))

## Generate abbreveation for gender and race in one column

In [ ]:
labels = {
    'F_AIAN'   : "('F', 'AMERICAN INDIAN/ALASKA NATIVE')",
    'F_AS'     : "('F', 'ASIAN')",
    'F_AS_AI'  : "('F', 'ASIAN - ASIAN INDIAN')",
    'F_AS_CH'  : "('F', 'ASIAN - CHINESE')",
    'F_AS_SEA' : "('F', 'ASIAN - SOUTH EAST ASIAN')",
    'F_BL_A'   : "('F', 'BLACK/AFRICAN')",
    'F_BL_AA'  : "('F', 'BLACK/AFRICAN AMERICAN')",
    'F_BL_CV'  : "('F', 'BLACK/CAPE VERDEAN')",
    'F_BL_CI'  : "('F', 'BLACK/CARIBBEAN ISLAND')",
    'F_HL'     : "('F', 'HISPANIC OR LATINO')",
    'F_HL_CO'  : "('F', 'HISPANIC/LATINO - COLUMBIAN')",
    'F_HL_DO'  : "('F', 'HISPANIC/LATINO - DOMINICAN')",
    'F_HL_GU'  : "('F', 'HISPANIC/LATINO - GUATEMALAN')",
    'F_HL_PR'  : "('F', 'HISPANIC/LATINO - PUERTO RICAN')",
    'F_HL_SA'  : "('F', 'HISPANIC/LATINO - SALVADORAN')",
    'F_PT'     : "('F', 'PORTUGUESE')",
    'F_W'      : "('F', 'WHITE')",
    'F_W_BR'   : "('F', 'WHITE - BRAZILIAN')",
    'F_W_EE'   : "('F', 'WHITE - EASTERN EUROPEAN')",
    'F_W_OE'   : "('F', 'WHITE - OTHER EUROPEAN')",
    'F_W_RU'   : "('F', 'WHITE - RUSSIAN')",
    'M_AIAN'   : "('M', 'AMERICAN INDIAN/ALASKA NATIVE')",
    'M_AS'     : "('M', 'ASIAN')",
    'M_AS_AI'  : "('M', 'ASIAN - ASIAN INDIAN')",
    'M_AS_CH'  : "('M', 'ASIAN - CHINESE')",
    'M_AS_SEA' : "('M', 'ASIAN - SOUTH EAST ASIAN')",
    'M_BL_A'   : "('M', 'BLACK/AFRICAN')",
    'M_BL_AA'  : "('M', 'BLACK/AFRICAN AMERICAN')",
    'M_BL_CV'  : "('M', 'BLACK/CAPE VERDEAN')",
    'M_BL_CI'  : "('M', 'BLACK/CARIBBEAN ISLAND')",
    'M_HL'     : "('M', 'HISPANIC OR LATINO')",
    'M_HL_DO'  : "('M', 'HISPANIC/LATINO - DOMINICAN')",
    'M_HL_GU'  : "('M', 'HISPANIC/LATINO - GUATEMALAN')",
    'M_HL_PR'  : "('M', 'HISPANIC/LATINO - PUERTO RICAN')",
    'M_HL_SA'  : "('M', 'HISPANIC/LATINO - SALVADORAN')",
    'M_PT'     : "('M', 'PORTUGUESE')",
    'M_W'      : "('M', 'WHITE')",
    'M_W_BR'   : "('M', 'WHITE - BRAZILIAN')",
    'M_W_EE'   : "('M', 'WHITE - EASTERN EUROPEAN')",
    'M_W_OE'   : "('M', 'WHITE - OTHER EUROPEAN')",
    'M_W_RU'   : "('M', 'WHITE - RUSSIAN')",
}
swapped_labels = {eval(v): k for k, v in labels.items()}

In [ ]:
merged_weighted['abbrev'] = merged_weighted.apply(lambda row: swapped_labels[(row['gender'], row['race'])], axis=1)

## Group by gender and race

In [ ]:
grouped = merged_weighted.groupby(['gender','race'])

In [ ]:
grouped.groups.keys()

In [ ]:
#grouped.get_group(('F', 'AMERICAN INDIAN/ALASKA NATIVE'))

### Sample 100 notes for every gender-race-subgroup

In [ ]:
len(age_dist_exact_prob) # number of different ages

In [ ]:
samples = {}

for group_name, group in grouped:
    
    key = f'{group_name}'
    sample_100 = pd.DataFrame(group.sample(n = 100, weights = 'weight', replace = False, random_state = 0))
    samples[key] = sample_100

sampled_data = pd.concat(samples, names = ['group']).reset_index()

Save a version of sampled_data with the all the original notes

In [ ]:
sampled_data_og = sampled_data.copy()

#### Check sampled data for transgender individuals

Look up sex in the note text

In [ ]:
def find_sex(text):
    sex = re.findall(r'Sex:\s*\S', text)
    sex = sex[0]
    sex = re.sub(r'Sex:\s*', '', sex)
    return (sex[0])

Replace trans individuals with other notes of the same race, gender and actual_age

In [ ]:
#add sex as a column
sampled_data['sex'] = sampled_data['text'].apply(find_sex)

#filter for rows where sex and gender don't match
trans = sampled_data[sampled_data['sex']!=sampled_data['gender']]

#drop trans notes from sampled_data
trans_hadm_id = trans['hadm_id'].to_list()
sampled_data = sampled_data[~sampled_data['hadm_id'].isin(trans_hadm_id)]

#sample new notes with same attributes as the dropped ones
trans_dict = trans.to_dict('records')
possible_patients = []

for row in trans_dict:
    group = grouped.get_group((row['gender'], row['race'])).copy()
    group['sex'] = group['text'].apply(find_sex)
    same_info = group[(group['sex'] == group['gender']) & (group['actual_age'] == row['actual_age'])].copy()
    same_info['group'] = (row['gender'], row['race'])
    possible_patients.append(same_info)

new_file_info = []

for df in possible_patients:
    sample = df.sample(n = 1, random_state = 0)
    sampled_data = pd.concat([sampled_data, sample]).reset_index()
    new_file_info.append((sample['abbrev'], sample['hadm_id']))

### Age Distribution for every subgroup

Boxplots

In [ ]:
sampled_data['race_abbrev'] = sampled_data['abbrev'].str[2:]

In [ ]:
sampled_data['race_wrapped'] = sampled_data['race'].str.replace(" ", "\n")
sampled_data['race_wrapped'] = sampled_data['race_wrapped'].str.replace("/", "/\n")
sampled_data['race_wrapped'] = sampled_data['race_wrapped'].str.replace("COLUMBIAN", "COLUM-\nBIAN")
sampled_data['race_wrapped'] = sampled_data['race_wrapped'].str.replace("GUATEMALAN", "GUATE-\nMALAN")
sampled_data['race_wrapped'] = sampled_data['race_wrapped'].str.replace("SALVADORAN", "SALVADO-\nRAN")
sampled_data['race_wrapped'] = sampled_data['race_wrapped'].str.replace("PORTUGUESE", "POR-\nTUGUESE")
sampled_data['race_wrapped'] = sampled_data['race_wrapped'].str.replace("EUROPEAN", "EURO-\nPEAN")
sampled_data['race_wrapped'] = sampled_data['race_wrapped'].str.replace("DOMINICAN", "DOMINI-\nCAN")

plt.figure(figsize = (12,4))
race_sample_violin = sns.violinplot(data = sampled_data, x = 'race_abbrev', y = 'actual_age', hue = 'gender')
plt.xlabel('Race', fontsize = 14)
plt.ylabel('Age', fontsize = 14)
plt.legend(loc = 'upper right', fontsize = 12)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 11)
plt.tight_layout()
race_sample_violin_fig = race_sample_violin.get_figure()
race_sample_violin_fig.savefig('/home/wite10/images/agedist_sample_violin_by_race.svg')
plt.show()

In [ ]:
for key in samples:
    
    print(key)
    
    age_dist = pd.DataFrame(samples[key]['actual_age'].value_counts()).reset_index()
    sns.barplot(x = 'actual_age', y = 'count', data = age_dist)
    plt.show()

Export notes:

In [ ]:
#for i in range(len(sampled_data)):
#    with open(f"grouped/{sampled_data['gender_race_abbrev'].iloc[i]}_{sampled_data['hadm_id'].iloc[i]}.txt", 'w') as file:
#        file.write(sampled_data['text'].iloc[i])

Export notes for swapped individuals of trans patients:

In [ ]:
#for info in new_file_info:
#    
#    abbrev = info[0].item()
#    hadm_id = info[1].item()
#
#    note = sampled_data[sampled_data['hadm_id'] == hadm_id]['text'].iloc[0]
#    
#    with open(f"grouped/new_copy/{abbrev}_{hadm_id}.txt", 'w') as file:
#        file.write(note)

Compute quartiles for merged (full filtered data set)

In [ ]:
merged['actual_age'].quantile([0,0.25,0.5,0.75,1])

Compute quartiles for each subgroup

In [ ]:
for key in samples:
    print(key)
    print(samples[key]['actual_age'].quantile([0,0.25,0.5,0.75,1]))

## Save demographics of each group
- was originally done before filtering out trans notes, that's why sampled_data_og is used

In [ ]:
sampled_data_attributes = sampled_data_og[['abbrev','hadm_id', 'gender', 'actual_age', 'race']]
sampled_data_attributes_new = sampled_data[['abbrev','hadm_id', 'gender', 'actual_age', 'race']]

#sampled_data_attributes.to_csv('demographics/sample_demographics/sample_demographics_trans.csv', index = False)
#sampled_data_attributes_new.to_csv('demographics/sample_demographics/sample_demographics_new.csv', index = False)

sampled_data_attributes = sampled_data_attributes.groupby('abbrev')

In [ ]:
#sampled_data_attributes.get_group('F_AIAN')

In [ ]:
#for name, group in sampled_data_attributes:
#    group[['hadm_id', 'gender', 'actual_age', 'race']].to_csv(f'demographics/demographics_by_subgroup/{name}_demographics.csv', index = False)

Save in random order for name and address generation

In [ ]:
sampled_data_shuffled = sampled_data_og.sample(frac=1, random_state = 0).reset_index(drop=True)
random_subgroups = np.array_split(sampled_data_shuffled, 41)

In [ ]:
#i = 1
#for group in random_subgroups:
#    group[['hadm_id', 'gender', 'actual_age', 'race']].to_csv(f'demographics/random_subgroups/subgroup_{i}.csv', index = False)
#    i += 1

Export text and readmission label for testing

In [ ]:
#sampled_data[['text','thirty_day_readmission']].to_csv('mimic_readmission.csv', index = False)

## Shift DOB, Admission Time and Discharge Time to present day

In [ ]:
#Shift date to given year
def shift_year(date, year):
    try:
        return date.replace(year = year)
    except ValueError: 
        #leap year
        return date.replace(year = year, day = 28)

In [ ]:
#Are admittime and dischtime in the same year?
same_year_visit = (sampled_data['admittime'].dt.year == sampled_data['dischtime'].dt.year)

#shift admission time to 2025
sampled_data.loc[:, 'admittime_today'] = sampled_data.loc[:, 'admittime'].apply(lambda x: shift_year(x,2025))

#shift dischtime according to admittime (2025 or 2026 if admittime and dischtime not in the same year)
sampled_data.loc[same_year_visit, 'dischtime_today'] = sampled_data.loc[same_year_visit, 'dischtime'].apply(lambda x: shift_year(x, 2025))
sampled_data.loc[~same_year_visit, 'dischtime_today'] = sampled_data.loc[~same_year_visit, 'dischtime'].apply(lambda x: shift_year(x, 2026))

In [ ]:
#sampled_data[['admittime', 'dischtime', 'admittime_today', 'dischtime_today']]

## Shift DOB according to present day

In [ ]:
sampled_data.loc[:, 'dob'] = (sampled_data.loc[:, 'dischtime_today']).apply(lambda x: x.year) - sampled_data.loc[:, 'actual_age']

In [ ]:
#sampled_data[['dob', 'actual_age']]

## Load names and addresses

#### Load generated data
- from sampled_data_og, therefore contains info associated with the swapped out patients

In [ ]:
demographic_subfiles = [f'demographics/demographics_random/subgroup_{i}.generated.csv' for i in range(1,42)]
demographics = pd.concat((pd.read_csv(file) for file in demographic_subfiles), ignore_index = True)

#### merge with other columns of sampled_data

In [ ]:
sampled_data_demographics = pd.merge(sampled_data, demographics[['hadm_id', 'name', 'address']], on = ['hadm_id'],
                                        how = 'left', validate = 'one_to_one')

Find matching names and addresses for swapped patients

In [ ]:
changed_notes = sampled_data_demographics[pd.isnull(sampled_data_demographics['name'])]
changed_notes = changed_notes.to_dict('records')

for pt in changed_notes:
    race = pt['race']
    age = pt['actual_age']
    gender = pt['gender']

    name = demographics[(demographics['race'] == race) & 
                        (demographics['actual_age'] == age) & 
                        (demographics['gender']== gender)
                        ]['name'].iloc[0]
    address = demographics[
                        (demographics['race'] == race) & 
                        (demographics['actual_age'] == age) & 
                        (demographics['gender']== gender)
                        ]['address'].iloc[0]

    pt['name'] = name
    pt['address'] = address

changed_notes = pd.DataFrame.from_records(changed_notes)

sampled_data_demographics = sampled_data_demographics[~pd.isnull(sampled_data_demographics['name'])]
sampled_data_demographics = pd.concat([sampled_data_demographics, changed_notes]).reset_index(drop = True)

In [ ]:
#sampled_data_demographics#[sampled_data_demographics['name']=='Andre Lewis']#['actual_age']

Extract important rows and group by race and gender

In [ ]:
demographics_check = sampled_data_demographics[['hadm_id', 'gender','actual_age', 'race', 'name', 'address', 'abbrev']]
demographics_check = demographics_check.groupby('abbrev')[['hadm_id', 'gender','actual_age', 'race', 'name', 'address']]

In [ ]:
#for gender_race, group in demographics_check:
#    group.to_csv(f'demographics/demographics_check/{gender_race}_demographics.csv', index = False)

In [ ]:
sampled_data['race'].unique()

## Save sampled_data_demographics for note_preparation

In [ ]:
#sampled_data_demographics.to_csv('demographics/sampled_data_demographics.csv', index = False)

In [ ]:
#demographics